# Meta-ness

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

One of the most basic objects in this approach are unevaluated *symbols*:

In [1]:
a

UndefVarError: UndefVarError: a not defined

In [2]:
:a  # "the symbol a"

:a

In [3]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [4]:
eval(:a)

UndefVarError: UndefVarError: a not defined

`a` must be defined for this to work:

In [5]:
a = 3

3

In [6]:
eval(:a)

3

The `eval` function takes an expression and evaluates it

Every name for a thing is a symbol:

In [7]:
:+, :sin

(:+, :sin)

In [8]:
typeof(:+)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [14]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [15]:
typeof(ex)

Expr

In [16]:
b = 7
eval(ex)

10

An expression is just a Julia object, so we can introspect (find out information about it):

In [17]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [18]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3


In [19]:
Meta.@dump x = 3

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3


In [ ]:
# ex.<TAB>

In [20]:
ex.head

:call

In [21]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [24]:
ex = :( sin(3a + 2b^2) )b

:(sin(3a + 2 * b ^ 2))

In [25]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 3
            3: Symbol a
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 2
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol b
                3: Int64 2


In [26]:
Meta.show_sexpr(ex)

(:call, :sin, (:call, :+, (:call, :*, 3, :a), (:call, :*, 2, (:call, :^, :b, 2))))

```
(sin (+ (* 3 a) (* 2 (^ b 2))))
```

In [27]:
ex.args

2-element Array{Any,1}:
 :sin             
 :(3a + 2 * b ^ 2)

In [28]:
typeof(ex.args[2])

Expr

In [29]:
ex.args[2].args

3-element Array{Any,1}:
 :+          
 :(3a)       
 :(2 * b ^ 2)

In [30]:
ex.args[1] = :cos

:cos

In [31]:
ex

:(cos(3a + 2 * b ^ 2))

In [32]:
blk = quote
    println("Hello")
end

quote
    #= In[32]:2 =#
    println("Hello")
end

In [33]:
dump(blk)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[32]
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "Hello"


In [34]:
eval(blk)

Hello


In [35]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :(#= In[32]:2 =#)  
 :(println("Hello"))
 :(println("AFTER"))

In [36]:
blk

quote
    #= In[32]:2 =#
    println("Hello")
    println("AFTER")
end

In [37]:
eval(blk)

Hello
AFTER


In [38]:
pushfirst!(blk.args, :(println("BEFORE")))

4-element Array{Any,1}:
 :(println("BEFORE"))
 :(#= In[32]:2 =#)   
 :(println("Hello")) 
 :(println("AFTER")) 

In [39]:
eval(blk)

BEFORE
Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [40]:
ex2 = quote
    y = 3
    z = sin(y+1)
end

quote
    #= In[40]:2 =#
    y = 3
    #= In[40]:3 =#
    z = sin(y + 1)
end

In [41]:
eval(ex2)

-0.7568024953079282

In [42]:
y, z

(3, -0.7568024953079282)

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [43]:
Meta.show_sexpr(ex2)

(:block,
  :(#= In[40]:2 =#),
  (:(=), :y, 3),
  :(#= In[40]:3 =#),
  (:(=), :z, (:call, :sin, (:call, :+, :y, 1)))
)

Another way of seeing the structure is with `dump`:

# Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code on the fly from within Julia*. This is known as *metaprogramming*: programming a program.

The name *macro* is given to a kind of "super-function" that takes a piece of code as an argument, and returns an altered piece of code. A macro is thus a very different kind of object than a standard function. [Although it can be thought of as a function in the mathematical sense of the word.]

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

In [ ]:
# @mac a b c ===>  code to execute

Although metaprogramming is possible in many languages, often by splicing strings together, Julia makes it considerably more natural, although not exactly easy – there seems to be something inherently tricky about the metaness of it.

Metaprogramming is useful in a variety of settings. The following are a few example use cases:
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [45]:
@time sin(10)

  0.000004 seconds (5 allocations: 176 bytes)


-0.5440211108893698

A trivial example of defining a macro is the following, which runs whatever code it is passed two times. The `$` sign is used to interpolate the value of the expression (similar to its usage for string interpolation):

In [47]:
macro twice(ex)
    quote
        $(esc(ex))
        $(esc(ex))
    end
end

@twice (macro with 1 method)

In [48]:
x = 0
@twice println(x += 1)

1
2


In [50]:
@twice println(x += 1)

5
6


We can see what effect the macro actually has using `Meta.@macroexpand`:

In [54]:
Meta.@macroexpand @twice println(x += 1)

quote
    #= In[47]:3 =#
    println(x += 1)
    #= In[47]:4 =#
    println(x += 1)
end

In [55]:
Meta.@macroexpand @time sin(10)

quote
    #= util.jl:154 =#
    local #6#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #8#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #7#val = sin(10)
    #= util.jl:157 =#
    #8#elapsedtime = (Base.time_ns)() - #8#elapsedtime
    #= util.jl:158 =#
    local #9#diff = (Base.GC_Diff)((Base.gc_num)(), #6#stats)
    #= util.jl:159 =#
    (Base.time_print)(#8#elapsedtime, (#9#diff).allocd, (#9#diff).total_time, (Base.gc_alloc_count)(#9#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #7#val
end

In [56]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [58]:
@mytime (sleep(1); "done")

1.0056240558624268 seconds elapsed


"done"

In [59]:
Meta.@macroexpand @mytime (sleep(1); "done")

quote
    #= In[56]:3 =#
    #16#t0 = (Main.time)()
    #= In[56]:4 =#
    #17#val = begin
            (Main.sleep)(1)
            #= In[59]:1 =#
            "done"
        end
    #= In[56]:5 =#
    #18#t1 = (Main.time)()
    #= In[56]:6 =#
    (Main.println)("$(#18#t1 - #16#t0) seconds elapsed")
    #= In[56]:7 =#
    #17#val
end

In [61]:
eval(Symbol("#16#t0"))

UndefVarError: UndefVarError: #16#t0 not defined

## Macro Hygiene (and lack thereof)

In [62]:
t1 = "the first terminator movie"
@mytime t1 *= " is the best"

UndefVarError: UndefVarError: #21#t1 not defined

In [63]:
Meta.@macroexpand @mytime t1 *= " is the best"

quote
    #= In[56]:3 =#
    #22#t0 = (Main.time)()
    #= In[56]:4 =#
    #23#val = (#24#t1 *= " is the best")
    #= In[56]:5 =#
    #24#t1 = (Main.time)()
    #= In[56]:6 =#
    (Main.println)("$(#24#t1 - #22#t0) seconds elapsed")
    #= In[56]:7 =#
    #23#val
end

Oops! We don't want the macro to rewrite our `t1` to `#42#t1`. To prevent this, we can call the `esc` function on it in our macro:

In [64]:
macro mytime(ex)
    quote
        t0 = time()
        val = $(esc(ex))
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [65]:
t1 = "the first terminator movie"
@mytime t1 *= " is the best"

5.0067901611328125e-6 seconds elapsed


"the first terminator movie is the best"

In [66]:
Meta.@macroexpand @mytime t1 *= " is the best"

quote
    #= In[64]:3 =#
    #28#t0 = (Main.time)()
    #= In[64]:4 =#
    #29#val = (t1 *= " is the best")
    #= In[64]:5 =#
    #30#t1 = (Main.time)()
    #= In[64]:6 =#
    (Main.println)("$(#30#t1 - #28#t0) seconds elapsed")
    #= In[64]:7 =#
    #29#val
end

In [67]:
macro set_a(val)
    :($(esc(:a)) = $(esc(val)))
end

@set_a (macro with 1 method)

In [68]:
Meta.@macroexpand @set_a 1.5

:(a = 1.5)

In [69]:
@set_a 1.5

1.5

In [70]:
a

1.5

----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

In [77]:
# implement `@until` here
macro until(cond, body)
    quote
        while true
            $(esc(body))
            $(esc(cond)) && break
        end
    end
end

@until (macro with 1 method)

In [79]:
let i = 0
    @until i ≥ 5 begin
        println(i)
        i += 1
    end
end

0
1
2
3
4


## Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro which is not exported is `@Base.Math.horner`, so in the current namespace, `horner` should be undefined:

In [80]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [81]:
x = 3
@horner(x, 2, 3, 4, 5)

182

In [84]:
p1(x) = 2 + 3x + 4x^2 + 5x^3

p1 (generic function with 1 method)

In [85]:
p1(3)

182

To see what the macro does to this call, we again use `macroexpand`:

In [86]:
Meta.@macroexpand @horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)

quote
    #33#t = x
    2 + #33#t * (3 + #33#t * (4 + #33#t * (5 + #33#t * (6 + #33#t * (7 + #33#t * (8 + #33#t * (9 + #33#t * 10)))))))
end

In [87]:
Meta.@macroexpand @Base.Math.horner(x, 2, 3, 4, 5)

quote
    #34#t = x
    (Base.Math.muladd)(#34#t, (Base.Math.muladd)(#34#t, (Base.Math.muladd)(#34#t, 5, 4), 3), 2)
end

In [88]:
?muladd

search: muladd



```
muladd(x, y, z)
```

Combined multiply-add: computes `x*y+z`, but allowing the add and multiply to be merged with each other or with surrounding operations for performance. For example, this may be implemented as an [`fma`](@ref) if the hardware supports it efficiently. The result can be different on different machines and can also be different on the same machine due to constant propagation or other optimizations. See [`fma`](@ref).

# Examples

```jldoctest
julia> muladd(3, 2, 1)
7

julia> 3 * 2 + 1
7
```


In [89]:
?fma

search: fma findmax findmax! @fastmath floatmax



```
fma(x, y, z)
```

Computes `x*y+z` without rounding the intermediate result `x*y`. On some systems this is significantly more expensive than `x*y+z`. `fma` is used to improve accuracy in certain algorithms. See [`muladd`](@ref).


In [90]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [91]:
@code_llvm f(0.1)


; Function f
; Location: In[90]:1
define double @julia_f_227132420(double) {
top:
; Function muladd; {
; Location: float.jl:404
  %1 = fmul contract double %0, 4.500000e+00
  %2 = fadd contract double %1, 3.400000e+00
  %3 = fmul contract double %2, %0
  %4 = fadd contract double %3, 2.300000e+00
  %5 = fmul contract double %4, %0
  %6 = fadd contract double %5, 1.200000e+00
;}
  ret double %6
}


In [92]:
@code_native f(0.1)

	.section	__TEXT,__text,regular,pure_instructions
; Function f {
; Location: In[90]:1
	movabsq	$5026123944, %rax       ## imm = 0x12B9490A8
; Function muladd; {
; Location: float.jl:404
	vmovsd	(%rax), %xmm1           ## xmm1 = mem[0],zero
	movabsq	$5026123952, %rax       ## imm = 0x12B9490B0
	vfmadd213sd	(%rax), %xmm0, %xmm1
	movabsq	$5026123960, %rax       ## imm = 0x12B9490B8
	vfmadd213sd	(%rax), %xmm0, %xmm1
	movabsq	$5026123968, %rax       ## imm = 0x12B9490C0
	vfmadd213sd	(%rax), %xmm1, %xmm0
;}
	retq
	nopl	(%rax)
;}
